In [19]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path, results_path
from preprocessing_modules import create_time_windows_with_labels, create_time_windows_with_metadata
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE
from tsai.models.ResNet import ResNet
from tsai.models import ResNet
from tsai.models.ResNet import ResNet
from fastai.metrics import accuracy
from fastai.data.core import DataLoaders
from fastai.learner import Learner
import csv

import torch
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")
from fastai.optimizer import SGD, Adam

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nwertheim/.netrc


/home/nwertheim/miniconda3/bin/python


In [20]:
# Define FCMAE Hyperparameters (DON'T CHANGE):
batch_size = 16
num_blocks = 5
kernel_size = 7
base_dim = 32
learning_rate_FCMAE = 0.00016938000495408888
input_dimension = 512

# Specify hyperparameters (CHANGE) 
learning_rate = 2.786356887938272e-07
optimizer_name = 'Adam' # or 'SGD'
data_subset = 40 # choose: 20, 40, 60, 80, 100

opt_func_map = {
'SGD': SGD,
'Adam': Adam,
}
opt_func = opt_func_map[optimizer_name]

Uncomment cel below if you want to save results

In [21]:
results_csv_path = os.path.join(results_path, "FCMAE_resnet_subset_runs.csv")

if not os.path.isfile(results_csv_path):
    with open(results_csv_path, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
            "run",
            "data_subset",
            "accuracy",
            "precision",
            "recall",
            "f1",
            "average_precision",
            "roc_auc"
        ])

In [22]:

'''MY DATA + class weights'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2_" + str(data_subset) + ".npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test_2.npy")
test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import SGD
from torch.optim import AdamW
from functools import partial
from fastai.optimizer import OptimWrapper
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from fastai.data.core import DataLoaders



from collections import Counter

train_windows, train_labels, train_meta = create_time_windows_with_metadata(train_data)
test_windows, test_labels, test_meta = create_time_windows_with_metadata(test_data)

# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
test_labels_tensor = test_labels_tensor.long()


print(train_labels_tensor.shape)

train_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_windows_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Test metadata for reference
test_meta = test_meta
print('batch size', batch_size)
dls = DataLoaders(train_loader, test_loader)


# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

297
192
{'record_name': 'Hopper-2022_02_08_15_07_31-0000010181-0011-chan3', 'signal': array([[14.23043527],
       [24.57069717],
       [17.99612535],
       ...,
       [-1.21857878],
       [ 0.82743019],
       [-2.60873858]]), 'fs': 20, 'preterm': 1}
torch.Size([3826])
batch size 16


In [23]:



# Load the pre-trained encoder
path = os.path.join(models_path, 'FCMAE_encoder_no_PCA_gpu_normalized_correct.pth')


# Load the full FCMAE model
fcmae = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size)


# Load the saved weights into the full model
state_dict = torch.load(path)
fcmae.load_state_dict(state_dict)

# Extract only the encoder from the FCMAE model
encoder = fcmae.encoder
# Set the encoder to evaluation mode and freeze the encoder parameters
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False  # Freeze the encoder

print(encoder)

Sequential(
  (0): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv1d(64, 128, kernel_size=(7,), stride=(2,), padding=(3,))
  (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv1d(128, 256, kernel_size=(7,), stride=(2,), padding=(3,))
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv1d(256, 512, kernel_size=(7,), stride=(2,), padding=(3,))
  (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
)


In [24]:
import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, average_precision_score, roc_auc_score,
    confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from fastai.learner import Learner
from fastai.optimizer import SGD
from fastai.metrics import accuracy
from tsai.data.core import DataLoaders
from fastai.callback.wandb import WandbCallback
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback


class FCMAEClassifier(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.classifier = ResNet(input_dimension, 2)  

    def forward(self, x):
        with torch.no_grad():  # freeze encoder
            x = self.encoder(x)
        return self.classifier(x)

# ----------------------------- Utility Functions -----------------------------
def seed_everything(seed):
    import random, os
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# ----------------------------- Experiment Setup -----------------------------
num_epochs = 10
num_runs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_train_losses = []
all_valid_losses = []
all_metrics = []

best_ap = -1
best_preds = None
best_targs = None
best_probs = None

all_pred_labels = []
all_pred_probs = []
best_run_index = -1

loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor) 

top_models = []  # List to store tuples of (AUC, model_state_dict, run_index)

for run in range(num_runs):
    print(f"\n🌀 Run {run+1}/{num_runs}")
    wandb.init(
        project=f"FCMAE-resnet-today-{data_subset}",
        name=f"resnet_final{run+1}",
        config={
            "architecture": "FCMAE ResNet",
            "epochs": num_epochs,
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "run": run + 1
        }
    )

    seed_everything(run + 30)

    model = FCMAEClassifier(encoder).to(device)

    learn = Learner(
        dls,
        model,
        loss_func=loss_func,
        opt_func=opt_func,
        metrics=accuracy,
        cbs=[
            EarlyStoppingCallback(monitor='valid_loss', patience=3),
            WandbCallback(log='all', log_model=True, log_preds=True)
        ]
    )
    learn.model.to(device)
    # cbs=[
    #         EarlyStoppingCallback(monitor='valid_loss', patience=3),
    #         SaveModelCallback(monitor='valid_loss', fname=f'best_fcmae_resnet_run_{run}'),
    #         WandbCallback(log='all', log_model=True, log_preds=True)
    #     ]
    # learn.fit_one_cycle(num_epochs, learning_rate, wd=1e-2)
    # no weight decay
    learn.fit_one_cycle(num_epochs, learning_rate)


    # Record losses
    losses = np.array(learn.recorder.losses)
    iters_per_epoch = len(dls.train)
    train_loss = losses[::2][:num_epochs]
    valid_loss = losses[1::2][:num_epochs]
    all_train_losses.append(train_loss)
    all_valid_losses.append(valid_loss)
    # Save epoch-wise train and validation loss to CSV
    loss_df = pd.DataFrame({
        "run_index": [run + 1] * num_epochs,  # 1-indexed run
        "epoch": list(range(1, num_epochs + 1)),
        "train_loss": train_loss,
        "valid_loss": valid_loss
    })

    # Append to CSV (create header only if first run)
    # loss_df.to_csv("fcmae_resnet_run_losses.csv", mode='a', header=(run == 0), index=False)

    # Predictions
    preds, targs = learn.get_preds(dl=dls.valid)

    # Convert logits to probabilities
    probs = torch.nn.functional.softmax(preds, dim=1)

    # Get predicted class labels (0 or 1)
    pred_labels = probs.argmax(dim=1)

    # Get probability of class 1
    pred_probs = probs[:, 1].cpu().numpy()

    # Convert true labels to numpy
    true_labels = targs.cpu().numpy()
    all_pred_labels.append(pred_labels.cpu().numpy())
    all_pred_probs.append(pred_probs)

    # Evaluation metrics
    acc = accuracy_score(true_labels, pred_labels)
    prec = precision_score(true_labels, pred_labels, zero_division=0)
    rec = recall_score(true_labels, pred_labels, zero_division=0)
    f1 = f1_score(true_labels, pred_labels, zero_division=0)
    ap = average_precision_score(true_labels, pred_probs)
    auc = roc_auc_score(true_labels, pred_probs)
    all_metrics.append([acc, prec, rec, f1, ap, auc])

    # # uncomment if you want to save results

    with open(results_csv_path, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([run + 1, data_subset, acc, prec, rec, f1, ap, auc])


    wandb.log({
        "final_accuracy": acc,
        "final_precision": prec,
        "final_recall": rec,
        "final_f1": f1,
        "final_average_precision": ap,
        "final_auc": auc
    })
    wandb.finish()

    if ap > best_ap:
        best_ap = ap
        best_preds = preds
        best_probs = pred_probs
        best_targs = true_labels
        best_run_index = run + 1  # runs are 1-indexed in your naming
    
    
    # Track top models for saving
    # top_models.append((auc, model.state_dict(), run))




🌀 Run 1/5


Could not gather input dimensions
WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'TensorDataset' object has no attribute 'items'


epoch,train_loss,valid_loss,accuracy,time
0,0.678144,0.596279,0.673958,00:03
1,0.675332,0.586264,0.646875,00:03
2,0.671043,0.592358,0.639583,00:03
3,0.671079,0.563688,0.643229,00:03
4,0.666614,0.611553,0.640104,00:03
5,0.664827,0.578497,0.641146,00:03
6,0.667244,0.598822,0.637500,00:03


Better model found at epoch 0 with valid_loss value: 0.5962789058685303.
Better model found at epoch 1 with valid_loss value: 0.5862643122673035.
Better model found at epoch 3 with valid_loss value: 0.5636880993843079.
No improvement since epoch 3: early stopping


accuracy,█▃▁▂▂▂▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
final_accuracy,▁
final_auc,▁
final_average_precision,▁
final_f1,▁
final_precision,▁
final_recall,▁
lr_0,▁▂▂▂▃▃▆▇▇█████████▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃
mom_0,███▇▇▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅


...


🌀 Run 5/5


Could not gather input dimensions
WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'TensorDataset' object has no attribute 'items'


epoch,train_loss,valid_loss,accuracy,time
0,0.713937,0.806761,0.212500,00:03
1,0.703730,0.802868,0.212500,00:03
2,0.714675,0.791947,0.396354,00:03
3,0.704580,0.734674,0.634375,00:03
4,0.705398,0.763510,0.437500,00:03
5,0.692930,0.730008,0.509896,00:03
6,0.683591,0.734581,0.497396,00:03
7,0.689925,0.759318,0.449479,00:03
8,0.688703,0.733313,0.508854,00:03


Better model found at epoch 0 with valid_loss value: 0.8067611455917358.
Better model found at epoch 1 with valid_loss value: 0.8028684854507446.
Better model found at epoch 2 with valid_loss value: 0.7919467687606812.
Better model found at epoch 3 with valid_loss value: 0.7346735000610352.
Better model found at epoch 5 with valid_loss value: 0.7300083637237549.
No improvement since epoch 5: early stopping


accuracy,▁▁▄█▅▆▆▅▆
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
final_accuracy,▁
final_auc,▁
final_average_precision,▁
final_f1,▁
final_precision,▁
final_recall,▁
lr_0,▁▁▁▁▂▅▆▇▇▇██████████▇▇▇▇▆▆▆▅▅▄▃▃▃▂▂▁▁▁▁▁
mom_0,███▇▇▅▅▅▃▃▁▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█


In [25]:
# epochs = np.arange(1, num_epochs + 1)
# plt.figure(figsize=(10, 5))

# mean_train = all_train_losses.mean(axis=0)
# std_train = all_train_losses.std(axis=0)
# plt.plot(epochs, mean_train, label='Train Loss')
# plt.fill_between(epochs, mean_train - std_train, mean_train + std_train, alpha=0.3)

# mean_valid = all_valid_losses.mean(axis=0)
# std_valid = all_valid_losses.std(axis=0)
# plt.plot(epochs, mean_valid, label='Valid Loss')
# plt.fill_between(epochs, mean_valid - std_valid, mean_valid + std_valid, alpha=0.3)

# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Average Loss Curves Over Five Runs')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [26]:
class FCMAEClassifier(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.classifier = ResNet(input_dimension, 2)  

    def forward(self, x):
        with torch.no_grad():  # freeze encoder
            x = self.encoder(x)
        return self.classifier(x)





In [27]:
# # ----------------------------- Post-Processing -----------------------------
# all_train_losses = np.stack(all_train_losses)
# all_valid_losses = np.stack(all_valid_losses)
# all_metrics = np.stack(all_metrics)
# print(f"\n🏆 Best model was from run {best_run_index} with AP score of {best_ap:.4f}")

# # Average + std metrics
# metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AP', 'AUC']
# mean_metrics = all_metrics.mean(axis=0)
# std_metrics = all_metrics.std(axis=0)

# print("\n📊 Final Evaluation:")
# for name, mean, std in zip(metric_names, mean_metrics, std_metrics):
#     print(f"{name}: {mean:.4f} ± {std:.4f}")

# # Save predictions
# pred_labels = best_preds.argmax(dim=1).cpu().numpy()
# pred_probs = torch.softmax(best_preds, dim=1)[:, 1].cpu().numpy()

# # Assuming `test_meta` has the record_name and window_index
# window_ids = [entry['record_name'] + '-' + str(entry['window_index']) for entry in test_meta]
# final_df = pd.DataFrame({
#     'window_id': window_ids,
#     'best_pred_label': pred_labels,
#     'best_pred_prob': pred_probs,
#     'target': best_targs
# })

# for run_idx, (labels, probs) in enumerate(zip(all_pred_labels, all_pred_probs), start=1):
#     final_df[f'pred_label{run_idx}'] = labels
#     final_df[f'pred_prob{run_idx}'] = probs

# # Sort by AUC and keep top 5
# # top_models_sorted = sorted(top_models, key=lambda x: x[0], reverse=True)[:5]

# # Save the top 5 models
# # for idx, (auc, state_dict, run_idx) in enumerate(top_models_sorted, start=1):
# #     save_path = f"top_model_100_{idx}_run_{run_idx+1}_auc_{auc:.4f}.pth"
# #     torch.save(state_dict, save_path)
# #     print(f"✅ Saved: {save_path}")


# # uncomment if you want to save results 
# final_df.to_csv("results_FCMAE_resnet_all_runs.csv", index=False)

# # ----------------------------- Plot Loss Curves -----------------------------
# epochs = np.arange(1, num_epochs + 1)
# plt.figure(figsize=(10, 5))

# mean_train = all_train_losses.mean(axis=0)
# std_train = all_train_losses.std(axis=0)
# plt.plot(epochs, mean_train, label='Train Loss')
# plt.fill_between(epochs, mean_train - std_train, mean_train + std_train, alpha=0.3)

# mean_valid = all_valid_losses.mean(axis=0)
# std_valid = all_valid_losses.std(axis=0)
# plt.plot(epochs, mean_valid, label='Valid Loss')
# plt.fill_between(epochs, mean_valid - std_valid, mean_valid + std_valid, alpha=0.3)

# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Average Loss Curves Over Five Runs')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# # ----------------------------- Plot Metrics -----------------------------
# plt.figure(figsize=(10, 5))
# plt.bar(metric_names, mean_metrics, yerr=std_metrics, capsize=5)
# plt.title('Average Performance Over Five Runs')
# plt.ylabel('Score')
# plt.ylim(0, 1)
# plt.grid(axis='y')
# plt.tight_layout()
# plt.show()

# # ----------------------------- Confusion Matrix -----------------------------
# cm = confusion_matrix(best_targs, pred_labels)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
#             xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
# plt.title('Best Performing Model Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()
